# CKY parser with constituent copying

This parser was written by Floris. It implements copying by marking every rule as a copy rule or a non-copy rule. Rules are in Chomsky Normal Form. Copy rules apply only if both daughters generate the same string.

In [ ]:
import cky_constituent_copy as ckypy
from IPython.display import Image
from IPython.display import display

We make a toy grammar. None of these rules are copy rules.

In [ ]:
grammar_bare = [("S",  [(["NP","VP"])]),
               ("VP", [(["VP","PP"]),
                       (["V","NP"]),
                       (["eats"])]),
               ("PP", [(["P","NP"])]),
               ("NP", [(["NP","PP"]),
                       (["Det","N"]),
                       (["she"])]),
               ("V" , [(["eats"])]),
               ("P" , [(["with"])]),
               ("N" , [(["fish"]),
                       (["fork"])]),
               ("Det",[(["a"])])]

Add copy markers

In [ ]:
grammar_copy = ckypy.mark_no_copy_rules(grammar_bare)

In [ ]:
print(ckypy.grammar2string_copy(grammar_copy))

A second grammar that generates full binary trees

In [ ]:
grammar_ambig = [("S",[(["S","S"],False),(["a"],False)])]
grammar_ambig_probs = [("S",[(["S","S"],False,0.5),(["a"],False,0.5)])]

In [ ]:
print(ckypy.grammar2string_copy(grammar_ambig))

A third grammar with copying

In [ ]:
grammar_ambig_copy = [("S",[(["S","S"],False),(["S","S"],True),(["a"],False)])]
grammar_ambig_copy_probs = [("S",[(["S","S"],False,0.3),(["S","S"],True,0.3),(["a"],False,0.4)])]

In [ ]:
print(ckypy.grammar2string_copy(grammar_ambig_copy))

## Let's run some examples

In [ ]:
s = "she eats a fish with a fork".split(" ")

In [ ]:
g = grammar_copy

Parse s with g

In [ ]:
chart,backpointers = ckypy.parse(s,g)

In [ ]:
ckypy.print_chart(chart)

In [ ]:
ckypy.print_backpointers(backpointers)

Collect the trees

In [ ]:
parses = ckypy.collect_trees(0,len(s),"S",chart,backpointers,g,s)

Print the trees to files

In [ ]:
for i,parse in enumerate(parses):    
    ckypy.tree_to_png(parse,"parse_%i.png"%i)

In [ ]:
x=Image(filename='parse_0.png')
y=Image(filename='parse_1.png')
display(x,y)

Count the trees

In [ ]:
ckypy.n_parses("S",chart,backpointers,g,s)

Calculate probabilities

In [ ]:
grammar_copy_probs = [("S",  [(["NP","VP"],False,1.)]),
           ("VP", [(["VP","PP"],False,0.25),
                   (["V","NP"],False,0.5),
                   (["eats"],False,0.25)]),
           ("PP", [(["P","NP"],False,1.)]),
           ("NP", [(["NP","PP"],False,0.3),
                   (["Det","N"],False,0.4),
                   (["she"],False,0.3)]),
           ("V" , [(["eats"],False,1.)]),
           ("P" , [(["with"],False,1.)]),
           ("N" , [(["fish"],False,0.4),
                   (["fork"],False,0.6)]),
           ("Det",[(["a"],False,1.)])]

In [ ]:
g_probs = ckypy.make_rule_probs(grammar_copy_probs)

In [ ]:
print(ckypy.grammar2string_copy_probs(grammar_copy_probs))


In [ ]:
(probs,s_prob)=ckypy.probability("S",chart,backpointers,g,s,g_probs)

The log probability of the sentence is the second element

In [ ]:
s_prob

Let's un-log it just to get a look

In [ ]:
ckypy.np.exp(s_prob)

## Full binary tree grammars

### Using the binary tree grammar without copying

In [ ]:
g = grammar_ambig

In [ ]:
s = ["a"]*3

In [ ]:
chart,backpointers = ckypy.parse(s,g)

In [ ]:
ckypy.print_chart(chart)

In [ ]:
ckypy.print_backpointers(backpointers)

In [ ]:
ckypy.n_parses("S",chart,backpointers,g,s)

In [ ]:
parses = ckypy.collect_trees(0,len(s),"S",chart,backpointers,g,s)

In [ ]:
for i,parse in enumerate(parses):    
    ckypy.tree_to_png(parse,"parse_%i.png"%i)

In [ ]:
x=Image(filename='parse_0.png')
y=Image(filename='parse_1.png')
display(x,y)

Probability of sentence

In [ ]:
g_probs = ckypy.make_rule_probs(grammar_ambig_probs)

In [ ]:
(probs,s_prob)=ckypy.probability("S",chart,backpointers,g,s,g_probs)

The log probability of the sentence is the second element

In [ ]:
s_prob

Let's un-log it just to get a look

In [ ]:
ckypy.np.exp(s_prob)

### Using the binary tree grammar with copying

In [ ]:
g = grammar_ambig_copy

In [ ]:
s = ["a"]*3

In [ ]:
chart,backpointers = ckypy.parse(s,g)

In [ ]:
ckypy.print_chart(chart)

In [ ]:
ckypy.print_backpointers(backpointers)

In [ ]:
ckypy.n_parses("S",chart,backpointers,g,s)

In [ ]:
parses = ckypy.collect_trees(0,len(s),"S",chart,backpointers,g,s)

In [ ]:
for i,parse in enumerate(parses):    
    ckypy.tree_to_png(parse,"parse_%i.png"%i)

In [ ]:
x=Image(filename='parse_0.png')
y=Image(filename='parse_1.png')
z=Image(filename='parse_2.png')
w=Image(filename='parse_3.png')


display(x,y,z,w)

Probability of sentence

In [ ]:
g_probs = ckypy.make_rule_probs(grammar_ambig_copy_probs)

In [ ]:
(probs,s_prob)=ckypy.probability("S",chart,backpointers,g,s,g_probs)

The log probability of the sentence is the second element

In [ ]:
s_prob

Let's un-log it just to get a look

In [ ]:
ckypy.np.exp(s_prob)